In [1]:
import fs from "node:fs";
import { dirname } from "node:path";
import { exec } from "node:child_process";

import * as Plot from "npm:@observablehq/plot";
import * as d3 from "npm:d3";
import { JSDOM } from "npm:jsdom";
import * as pl from "npm:nodejs-polars";
import { readCSV } from "npm:nodejs-polars";
import puppeteer from "npm:puppeteer";


In [2]:
function uniformSample<D extends pl.DataFrame>(df: D, targetSize: number = 1000): D {
  if (df.height <= targetSize) return df;

  const sampledColumns = df.columns.map(() => []);
  const ratio = df.height / targetSize;

  for (let i = 0; i < targetSize; i++) {
    const index = Math.round(i * ratio);
    const row = df.row(index);
    for (let j = 0; j < df.width; j++) sampledColumns[j].push(row[j]);
  }

  return pl.DataFrame(Object.fromEntries(sampledColumns.map((col, i) => [df.columns[i], col])));
}


In [3]:
async function savePNG(
  plot: Plot.Plot,
  path: string,
  options: {
    /** Padding around the plot in pixels */
    padding?: number;
  } = {}
) {
  const { padding = 0 } = options;

  const browser = await puppeteer.launch({
    headless: true,
    args: ["--no-sandbox", "--disable-setuid-sandbox"],
  });

  const page = await browser.newPage();

  await page.setViewport({
    // Simulate 4K resolution
    width: 3840,
    height: 2160,
    deviceScaleFactor: 2, // High DPI for better quality
  });

  const htmlContent =
    "<!DOCTYPE html>\n" +
    '<html lang="en">\n' +
    "  <head>\n" +
    '    <meta charset="UTF-8" />\n' +
    '    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n' +
    "    <style>\n" +
    "      body { margin: 0; }\n" +
    "      h2 { margin-top: 0; }\n" +
    "    </style>\n" +
    "  </head>\n" +
    "  <body>\n    " +
    plot.outerHTML.replace("max-width: initial;", `width: fit-content; padding: ${padding}px;`) +
    "\n  </body>\n" +
    "</html>";
  await page.setContent(htmlContent);

  // Capture the figure element
  const figureElement = (await page.$("figure")) || (await page.$("svg"));

  if (!figureElement) throw new Error("Figure element not found");

  const boundingBox = await figureElement.boundingBox();

  if (!boundingBox) throw new Error("Could not get figure bounding box");

  if (dirname(path) !== ".") await fs.promises.mkdir(dirname(path), { recursive: true });

  // Screenshot
  await figureElement.screenshot({
    path,
    type: "png",
    omitBackground: false, // Include background
  });

  await browser.close();
}

async function savePDF(
  plot: Plot.Plot,
  path: string,
  options: {
    /** Padding around the plot in pixels */
    padding?: number;
    /** Paper format */
    format?: "A4" | "A3" | "A5" | "Letter" | "Legal" | "Tabloid";
    printBackground?: boolean;
  } = {}
) {
  const { padding = 0, format = "A4", printBackground = true } = options;

  const browser = await puppeteer.launch({
    headless: true,
    args: ["--no-sandbox", "--disable-setuid-sandbox"],
  });

  const page = await browser.newPage();

  const htmlContent =
    "<!DOCTYPE html>\n" +
    '<html lang="en">\n' +
    "  <head>\n" +
    '    <meta charset="UTF-8" />\n' +
    '    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n' +
    "    <style>\n" +
    "      body { margin: 0; font-family: 'Times New Roman', serif; }\n" +
    "      h2 { margin-top: 0; }\n" +
    "      figure { margin: 0; }\n" +
    "    </style>\n" +
    "  </head>\n" +
    "  <body>\n    " +
    plot.outerHTML.replace("max-width: initial;", `width: fit-content; padding: ${padding}px;`) +
    "\n  </body>\n" +
    "</html>";
  await page.setContent(htmlContent);

  await page.pdf({
    path,
    format,
    printBackground,
    preferCSSPageSize: true,
  });

  if (dirname(path) !== ".") await fs.promises.mkdir(dirname(path), { recursive: true });

  await browser.close();
}

async function saveEPS(
  plot: Plot.Plot,
  path: string,
  options: {
    /** Padding around the plot in pixels */
    padding?: number;
    printBackground?: boolean;
  } = {}
) {
  const { padding = 0, printBackground = true } = options;

  const tempFilePath = path.replace(/\.eps$/, "") + "-" + Date.now() + ".pdf";

  const browser = await puppeteer.launch({
    headless: true,
    args: ["--no-sandbox", "--disable-setuid-sandbox"],
  });

  const page = await browser.newPage();

  const htmlContent =
    "<!DOCTYPE html>\n" +
    '<html lang="en">\n' +
    "  <head>\n" +
    '    <meta charset="UTF-8" />\n' +
    '    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n' +
    "    <style>\n" +
    "      body { margin: 0; font-family: 'Times New Roman', serif; }\n" +
    "      h2 { margin-top: 0; }\n" +
    "      figure { margin: 0; }\n" +
    "    </style>\n" +
    "  </head>\n" +
    "  <body>\n    " +
    plot.outerHTML.replace("max-width: initial;", `width: fit-content; padding: ${padding}px;`) +
    "\n  </body>\n" +
    "</html>";
  await page.setContent(htmlContent);

  const figure = (await page.$("figure")) || (await page.$("svg"));
  if (!figure) throw new Error("Figure element not found");
  const box = await figure.boundingBox();
  if (!box) throw new Error("Could not compute bounding box");

  if (dirname(path) !== ".") await fs.promises.mkdir(dirname(path), { recursive: true });

  await page.pdf({
    path: tempFilePath,
    printBackground,
    width: `${Math.ceil(box.width)}px`,
    height: `${Math.ceil(box.height)}px`,
  });

  await browser.close();

  try {
    await new Promise((resolve, reject) => {
      exec(`inkscape "${tempFilePath}" --export-filename="${path}"`, (error) => {
        if (error) reject(error);
        else resolve(null);
      });
    });
  } catch (error) {
    console.error("inkscape not found. Please install Inkscape");
    console.error("Ubuntu/Debian: sudo apt install inkscape");
    console.error("macOS: brew install inkscape");
  }

  await fs.promises.rm(tempFilePath, { force: true });
}


In [4]:
const df = readCSV(fs.readFileSync("meta_large_miss_ratio.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("W-TinyLFU_CMS").as("CMS"),
  pl.col("W-TinyLFU_ADA").as("Ada"),
  pl.col("W-TinyLFU_EVO_TUNING_ONLY").as("Evolving (Tuning Only)"),
  pl.col("W-TinyLFU_EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.089, 0.154],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_meta_large.eps");
Plot.plot({
  ...options,
  title: "Meta KV (Large) - Miss Ratio V.S. Alpha",
});


Meta KV (Large) - Miss Ratio V.S. Alpha Ada CMS Evolving Evolving (Tuning Only) 0.09 0.10 0.11 0.12 0.13 0.14 0.15 Miss Ratio 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [5]:
const df = readCSV(fs.readFileSync("meta_small_miss_ratio.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("W-TinyLFU_CMS").as("CMS"),
  pl.col("W-TinyLFU_ADA").as("Ada"),
  pl.col("W-TinyLFU_EVO_TUNING_ONLY").as("Evolving (Tuning Only)"),
  pl.col("W-TinyLFU_EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.158, 0.275],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_meta_small.eps");
Plot.plot({
  ...options,
  title: "Meta KV Trace (Small) - Miss Ratio V.S. Alpha",
});


Meta KV Trace (Small) - Miss Ratio V.S. Alpha Ada CMS Evolving Evolving (Tuning Only) 0.16 0.18 0.20 0.22 0.24 0.26 Miss Ratio 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [6]:
const df = readCSV(fs.readFileSync("msr_large_miss_ratio.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("W-TinyLFU_CMS").as("CMS"),
  pl.col("W-TinyLFU_ADA").as("Ada"),
  pl.col("W-TinyLFU_EVO_TUNING_ONLY").as("Evolving (Tuning Only)"),
  pl.col("W-TinyLFU_EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.0015, 0.0315],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 108,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_msr_large.eps");
Plot.plot({
  ...options,
  title: "MSR Trace (Large) - Miss Ratio V.S. Alpha",
});


MSR Trace (Large) - Miss Ratio V.S. Alpha Ada CMS Evolving Evolving (Tuning Only) 0.005 0.010 0.015 0.020 0.025 0.030 Miss Ratio 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [7]:
const df = readCSV(fs.readFileSync("msr_small_miss_ratio.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("W-TinyLFU_CMS").as("CMS"),
  pl.col("W-TinyLFU_ADA").as("Ada"),
  pl.col("W-TinyLFU_EVO_TUNING_ONLY").as("Evolving (Tuning Only)"),
  pl.col("W-TinyLFU_EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, missRatio]) => ({ alpha: record.alpha, baseline, missRatio }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -40,
  },
  y: {
    label: "Miss Ratio",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.405, 0.525],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "missRatio",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 64 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/miss_ratio_msr_small.eps");
Plot.plot({
  ...options,
  title: "MSR Trace (Small) - Miss Ratio V.S. Alpha",
});


MSR Trace (Small) - Miss Ratio V.S. Alpha Ada CMS Evolving Evolving (Tuning Only) 0.42 0.44 0.46 0.48 0.50 0.52 Miss Ratio 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 (Initial) Alpha

In [8]:
const df = readCSV(fs.readFileSync("hm_large_coverage.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("CMS"),
  pl.col("ADA").as("Ada"),
  pl.col("EVO_TUNING_ONLY").as("Evolving (Tuning Only)"),
  pl.col("EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, trendingCoverage]) => ({ alpha: record.alpha, baseline, trendingCoverage }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "Trending Coverage",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.22, 0.67],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "trendingCoverage",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "trendingCoverage",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 84,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/coverage_hm_large.eps");
Plot.plot({
  ...options,
  title: "H&M Trending (K=5000) - Coverage V.S. Alpha",
});


H&M Trending (K=5000) - Coverage V.S. Alpha Ada CMS Evolving Evolving (Tuning Only) 0.3 0.4 0.5 0.6 Trending Coverage 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 2,000 5,000 10,000 (Initial) Alpha

In [9]:
const df = readCSV(fs.readFileSync("hm_small_coverage.csv", "utf-8")).select(
  pl.col("alpha"),
  pl.col("CMS"),
  pl.col("ADA").as("Ada"),
  pl.col("EVO_TUNING_ONLY").as("Evolving (Tuning Only)"),
  pl.col("EVO").as("Evolving")
);
const data = df.toRecords().flatMap((record) =>
  Object.entries(record)
    .filter(([col]) => col !== "alpha")
    .map(([baseline, trendingCoverage]) => ({ alpha: record.alpha, baseline, trendingCoverage }))
);
const options = {
  symbol: { legend: true },
  x: {
    label: "(Initial) Alpha",
    domain: data.map((d) => d.alpha), // Preserve the order of alphas
    tickRotate: -45,
  },
  y: {
    label: "Trending Coverage",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0.03, 0.27],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.lineY(data, {
      x: "alpha",
      y: "trendingCoverage",
      stroke: "baseline",
      strokeWidth: 4,
    }),
    Plot.dot(data, {
      x: "alpha",
      y: "trendingCoverage",
      stroke: "baseline",
      symbol: "baseline",
      strokeWidth: 3,
      r: 8,
    }),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 80,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 72 },
});
d3.select(plot)
  .select("div")
  .style("font-size", "20px")
  .style("margin-left", "60px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/coverage_hm_small.eps");
Plot.plot({
  ...options,
  title: "H&M Trending (K=1000) - Coverage V.S. Alpha",
});


H&M Trending (K=1000) - Coverage V.S. Alpha Ada CMS Evolving Evolving (Tuning Only) 0.05 0.10 0.15 0.20 0.25 Trending Coverage 0.1 0.2 0.5 1 2 5 10 20 50 100 200 500 1,000 2,000 5,000 10,000 (Initial) Alpha

In [10]:
const queryLatency = readCSV(fs.readFileSync("msr_small_query_throughput.csv", "utf-8"))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_TUNING_ONLY").as("Evolving-TO"),
    pl.col("W-TinyLFU_EVO").as("Evolving")
  )
  .filter(pl.col("alpha").eq(1))
  .drop("alpha")
  .toRecords()[0];
const updateLatency = readCSV(fs.readFileSync("msr_small_update_throughput.csv", "utf-8"))
  .select(
    pl.col("alpha"),
    pl.col("W-TinyLFU_CMS").as("CMS"),
    pl.col("W-TinyLFU_ADA").as("Ada"),
    pl.col("W-TinyLFU_EVO_TUNING_ONLY").as("Evolving-TO"),
    pl.col("W-TinyLFU_EVO").as("Evolving")
  )
  .filter(pl.col("alpha").eq(1))
  .drop("alpha")
  .toRecords()[0];
const data = Object.keys(queryLatency).flatMap((baseline) => [
  { baseline, type: "Query", throughput: 1.0 / queryLatency[baseline] / 1_000_000 },
  { baseline, type: "Update", throughput: 1.0 / updateLatency[baseline] / 1_000_000 },
]);
const options = {
  color: { legend: true },
  x: { label: null },
  y: {
    label: "Throughput (MOps)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0, 23],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.barY(data, {
      x: "baseline",
      fill: "type",
      opacity: 0, // Only used to add legend
    }),
    Plot.barY(
      data.filter((d) => d.type === "Query"),
      {
        x: "baseline",
        y: "throughput",
        insetRight: 62,
        fill: "#4269d0",
        stroke: "black",
        strokeWidth: 3,
      }
    ),
    Plot.barY(
      data.filter((d) => d.type === "Update"),
      {
        x: "baseline",
        y: "throughput",
        insetLeft: 62,
        fill: "#f0b018",
        stroke: "black",
        strokeWidth: 3,
      }
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 76,
  style: { fontSize: "24px" },
  x: { ...options.x },
});
d3.select(plot)
  .select("div")
  .style("font-size", "24px")
  .style("margin-left", "220px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/throughput_msr.eps");
Plot.plot({
  ...options,
  title: "Query & Update Throughput on MSR",
});


Query & Update Throughput on MSR Query Update 0 5 10 15 20 Throughput (MOps) Ada CMS Evolving Evolving-TO

In [11]:
const queryLatency = readCSV(fs.readFileSync("hm_small_query_throughput.csv", "utf-8"))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_TUNING_ONLY").as("Evolving-TO"),
    pl.col("EVO").as("Evolving")
  )
  .filter(pl.col("alpha").eq(1))
  .drop("alpha")
  .toRecords()[0];
const updateLatency = readCSV(fs.readFileSync("hm_small_update_throughput.csv", "utf-8"))
  .select(
    pl.col("alpha"),
    pl.col("CMS"),
    pl.col("ADA").as("Ada"),
    pl.col("EVO_TUNING_ONLY").as("Evolving-TO"),
    pl.col("EVO").as("Evolving")
  )
  .filter(pl.col("alpha").eq(1))
  .drop("alpha")
  .toRecords()[0];
const data = Object.keys(queryLatency).flatMap((baseline) => [
  { baseline, type: "Query", throughput: 1.0 / queryLatency[baseline] / 1_000_000 },
  { baseline, type: "Update", throughput: 1.0 / updateLatency[baseline] / 1_000_000 },
]);
const options = {
  color: { legend: true },
  x: { label: null },
  y: {
    label: "Throughput (MOps)",
    labelAnchor: "center",
    labelArrow: false,
    ticks: 5,
    domain: [0, 23.9],
  },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.barY(data, {
      x: "baseline",
      fill: "type",
      opacity: 0, // Only used to add legend
    }),
    Plot.barY(
      data.filter((d) => d.type === "Query"),
      {
        x: "baseline",
        y: "throughput",
        insetRight: 62,
        fill: "#4269d0",
        stroke: "black",
        strokeWidth: 3,
      }
    ),
    Plot.barY(
      data.filter((d) => d.type === "Update"),
      {
        x: "baseline",
        y: "throughput",
        insetLeft: 62,
        fill: "#f0b018",
        stroke: "black",
        strokeWidth: 3,
      }
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 76,
  style: { fontSize: "24px" },
  x: { ...options.x },
});
d3.select(plot)
  .select("div")
  .style("font-size", "24px")
  .style("margin-left", "220px")
  .style("margin-bottom", "-10px");
await saveEPS(plot, "../output/figures/throughput_hm.eps");
Plot.plot({
  ...options,
  title: "Query & Update Throughput on H&M",
});


Query & Update Throughput on H&M Query Update 0 5 10 15 20 Throughput (MOps) Ada CMS Evolving Evolving-TO

In [12]:
const df = readCSV(fs.readFileSync("msr.alpha_1.trace.csv", "utf-8")).select(
  pl.col("Objective").as("hitRate"),
  pl.col("Parameter").as("alpha")
);
console.log("Trace length:", df.height);
console.log(df.describe().toString());
const data = uniformSample(df, 1000)
  .toRecords()
  .map((d, i) => ({ index: i, ...d }));
const y2 = d3.scaleLog(
  d3.extent(data, (d) => d.alpha),
  d3.extent(data, (d) => 1 - d.hitRate)
);
const options = {
  x: { label: "Time", labelAnchor: "center" },
  marks: [
    Plot.frame({ strokeWidth: 4 }),
    Plot.gridX({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.gridY({ strokeDasharray: "2", strokeOpacity: 0.3, strokeWidth: 2 }),
    Plot.axisY({
      axis: "left",
      color: "hsl(207, 45%, 30%)",
      label: "Miss Ratio",
      labelAnchor: "center",
      labelArrow: false,
      ticks: 5,
    }),
    Plot.axisY(y2.ticks(), {
      anchor: "right",
      color: "hsl(0, 53%, 40%)",
      label: "Alpha",
      labelAnchor: "center",
      labelArrow: false,
      y: y2,
      tickFormat: (d) => {
        let s = y2.tickFormat()(d);
        if (s === "200m") s = "0.2";
        return s;
      },
    }),
    Plot.lineY(data, { x: "index", y: (d) => 1 - d.hitRate, stroke: "steelblue" }),
    Plot.dotY(
      data,
      Plot.mapY((D) => D.map(y2), { x: "index", y: (d) => d.alpha, r: 1, stroke: "indianred" })
    ),
  ],
  document: new JSDOM().window.document,
} satisfies Plot.PlotOptions;
const plot = Plot.plot({
  ...options,
  marginLeft: 96,
  marginBottom: 64,
  marginRight: 72,
  style: { fontSize: "24px" },
  x: { ...options.x, labelOffset: 56 },
});
d3.select(plot).select("div").style("font-size", "20px");
await saveEPS(plot, "../output/figures/adaptation_timeline_msr.eps");
Plot.plot({
  ...options,
  title: "MSR Trace - Adaptation Timeline",
});


Trace length: 1686


shape: (5, 3)
┌──────────┬──────────┬────────────┐
│ describe ┆ hitRate  ┆ alpha      │
│ ---      ┆ ---      ┆ ---        │
│ str      ┆ f64      ┆ f64        │
╞══════════╪══════════╪════════════╡
│ mean     ┆ 0.584586 ┆ 10.74801   │
│ std      ┆ 0.027546 ┆ 63.666335  │
│ min      ┆ 0.39207  ┆ 0.10975    │
│ max      ┆ 0.64029  ┆ 911.162756 │
│ median   ┆ 0.581185 ┆ 0.48626    │
└──────────┴──────────┴────────────┘


MSR Trace - Adaptation Timeline 0 200 400 600 800 Time → 0.40 0.45 0.50 0.55 0.60 Miss Ratio 0.2 1 2 10 20 100 200 Alpha <path d="M40,20L40.561,224.454L41.121,218.4L41.682,226.056L42.242,228.92L42.803,198.863L43.363,218.287L43.924,231.401L44.484,217.535L45.045,222.044L45.606,226.453L46.166,220.867L46.727,231.883L47.287,219.775L47.848,233.513L48.408,222.838L48.969,218.499L49.53,233.13L50.09,220.257L50.651,169.672L51.211,245.805L51.772,243.877L52.332,240.106L52.893,234.08L53.453,246.684L54.014,263.456L54.575,256.169L55.135,263.328L55.696,259.047L56.256,255.701L56.817,259.586L57.377,255.276L57.938,249.052L58.498,259.983L59.059,261.23L59.62,255.63L60.18,253.532L60.741,260.932L61.301,252.738L61.862,259.019L62.422,253.574L62.983,270.814L63.544,217.521L64.104,241.226L64.665,274.217L65.225,251.632L65.786,264.278L66.346,280.753L66.907,281.334L67.467,274.203L68.028,274.756L68.589,228.282L69.149,267.61L69.71,260.309L70.27,267.327L70.831,265.285L71.391,262.563L71.952,258.834L72.513,277.336L73.073,266.462L73.634,268.432L74.194,269.808L74.755,265.242L75.315,267.426L75.876,267.128L76.436,273.097L76.997,266.377L77.558,271.452L78.118,270.658L78.679,267.865L79.239,267.383L79.8,214.53L80.36,244.572L80.921,212.247L81.481,281.83L82.042,272.36L82.603,271.764L83.163,275.819L83.724,271.665L84.284,283.149L84.845,273.21L85.405,279.278L85.966,277.889L86.527,276.343L87.087,277.549L87.648,273.196L88.208,278.654L88.769,272.119L89.329,278.782L89.89,266.136L90.45,278.243L91.011,281.206L91.572,283.687L92.132,286.58L92.693,280.356L93.253,271.764L93.814,277.506L94.374,276.755L94.935,276.443L95.495,279.888L96.056,277.449L96.617,278.881L97.177,262.932L97.738,281.532L98.298,274.67L98.859,273.593L99.419,241.041L99.98,272.842L100.541,270.219L101.101,222.993L101.662,265.186L102.222,221.335L102.783,273.48L103.343,260.621L103.904,269.793L104.464,264.406L105.025,276.06L105.586,275.677L106.146,272.501L106.707,279.76L107.267,272.416L107.828,275.564L108.388,276.641L108.949,270.814L109.51,275.62L110.07,268.603L110.631,277.01L111.191,279.477L111.752,275.819L112.312,280.597L112.873,273.749L113.433,209.156L113.994,277.506L114.555,273.933L115.115,270.474L115.676,286.211L116.236,272.884L116.797,282.964L117.357,272.912L117.918,278.711L118.478,279.817L119.039,273.834L119.6,276.145L120.16,269.609L120.721,275.223L121.281,286.806L121.842,275.72L122.402,277.336L122.963,284.751L123.524,279.165L124.084,273.905L124.645,274.94L125.205,274.67L125.766,272.898L126.326,226.892L126.887,267.312L127.447,287.827L128.008,297.794L128.569,284.751L129.129,302.43L129.69,290.507L130.25,301.026L130.811,294.93L131.371,272.742L131.932,299.595L132.492,288.564L133.053,298.574L133.614,287.218L134.174,300.984L134.735,297.184L135.295,292.562L135.856,290.847L136.416,289.217L136.977,293.3L137.538,294.434L138.098,299.75L138.659,284.198L139.219,298.574L139.78,288.267L140.34,272.586L140.901,286.098L141.461,290.804L142.022,293.47L142.583,297.737L143.143,291.669L143.704,297.156L144.264,294.831L144.825,297.284L145.385,221.661L145.946,298.701L146.507,306.3L147.067,310.157L147.628,300.488L148.188,310.469L148.749,307.179L149.309,249.321L149.87,295.497L150.43,309.618L150.991,303.394L151.552,302.09L152.112,293.753L152.673,287.955L153.233,309.845L153.794,302.444L154.354,294.051L154.915,305.492L155.475,288.933L156.036,284.524L156.597,295.837L157.157,307.449L157.718,292.321L158.278,295.058L158.839,307.307L159.399,307.605L159.96,307.194L160.521,296.475L161.081,301.112L161.642,306.215L162.202,251.249L162.763,271.339L163.323,297.695L163.884,297.851L164.444,311.007L165.005,290.408L165.566,304.769L166.126,300.927L166.687,310.795L167.247,242.431L167.808,287.969L168.368,322.675L168.929,310.355L169.489,299.991L170.05,315.587L170.611,310.27L171.171,302.047L171.732,318.536L172.292,318.734L172.853,304.202L173.413,314.027L173.974,315.884L174.535,231.911L175.095,294.065L175.656,315.388L176.216,300.445L176.777,262.506L177.337,314.679L177.898,304.812L178.458,311.589L179.019,307.094L179.58,303.238L180.14,303.55L180.701,306.612L181.261,291.81